In [1]:
import matplotlib.pyplot as plt
import json
import os
import cv2
import time
import numpy as np
from csbdeep.utils import Path, normalize
from segmentation import MELC_Segmentation
import pickle
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import seaborn as sns
import umap


2023-06-13 17:23:52.282919: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-13 17:23:52.284134: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-13 17:23:52.309422: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-13 17:23:52.309835: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 17:23:52.682162: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Normalize expression w.r.t. control mean and std deviation
segment = "nuclei"
dfs = list()
fields_of_view = [f for f in os.listdir(f"./marker_expression_{segment}_results/") if f.endswith(".pkl")]

for fov in fields_of_view:
    df = pd.read_pickle(f"./marker_expression_{segment}_results/{fov}")  
    df["Field of View"] = fov.split(".")[0]
    df["Sample"] = fov.split(" ")[0]
    df["Group"] = "Case" if "ALS" in fov else "Control"
    temp = df.index
    df["Index"] = df.index
    df = df.set_index(["Field of View", "Index"])
    dfs.append(df)

expression_data = pd.concat(dfs)
y_cols = ["Group", "Sample"]
ys = expression_data[y_cols]
Xs = expression_data.drop(labels=y_cols, axis=1)

control = expression_data[expression_data["Group"] == "Control"].drop(labels=y_cols, axis=1)
control_mean = control.mean(axis=0)
control_std = control.std(axis=0)
Xs_normalized = (Xs - control_mean) / control_std

binary_df = (Xs_normalized > 0)
binary_df = binary_df.astype(int)
#binary_df.to_pickle(f"./binary_expression_{segment}_only_control_as_reference.pkl")  

In [4]:
binary_df.sum(axis=0)

CD11b-PE       42290
CD16-PE        33250
CD2-FITC       52459
CD26-PE        33263
CD29-FITC      47078
CD3-PE         50612
CD36-FITC      41288
CD38-PE        42934
CD4-PE         45723
CD45-PE        62136
CD45RA-PE      51843
CD56-PE        45250
CD57-PE        21515
CD7-PE         51617
CD8-PE         33285
HLA-DQ-FITC    54210
HLA-DR-PE      43764
NeuN-AF488     32154
PBS            11557
dtype: int64

In [5]:
# Normalize expression w.r.t. global mean and std deviation
segment = "nuclei"
dfs = list()
fields_of_view = [f for f in os.listdir(f"./marker_expression_{segment}_results/") if f.endswith(".pkl")]

for fov in fields_of_view:
    df = pd.read_pickle(f"./marker_expression_{segment}_results/{fov}")   
    df["Field of View"] = fov.split(".")[0]
    df["Sample"] = fov.split(" ")[0]
    df["Group"] = "Case" if "ALS" in fov else "Control"
    temp = df.index
    df["Index"] = df.index
    df = df.set_index(["Field of View", "Index"])
    dfs.append(df)

expression_data = pd.concat(dfs)
y_cols = ["Group", "Sample"]
ys = expression_data[y_cols]
Xs = expression_data.drop(labels=y_cols, axis=1)

Xs_normalized = StandardScaler().fit_transform(Xs)
df_normalized = pd.DataFrame(data=Xs, index=Xs.index, columns=Xs.columns)

binary_df = (df_normalized > 0)
binary_df = binary_df.astype(int)
#binary_df.to_pickle(f"./binary_expression_{segment}_all_samples_as_reference.pkl")  

In [6]:
binary_df.sum(axis=0)

CD11b-PE        68383
CD16-PE         53242
CD2-FITC       152956
CD26-PE         58166
CD29-FITC       91760
CD3-PE          92298
CD36-FITC       76513
CD38-PE         85173
CD4-PE          90341
CD45-PE        134836
CD45RA-PE       99763
CD56-PE        131928
CD57-PE         29977
CD7-PE          93734
CD8-PE          61438
HLA-DQ-FITC    161074
HLA-DR-PE       85686
NeuN-AF488      44185
PBS             16921
dtype: int64